<a href="https://colab.research.google.com/github/emil-freme/rflearnign-autobot/blob/main/Reinforcement_Learning_OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
!pip install gymnasium

In [91]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np

# Definição do ambiente
> https://gymnasium.farama.org/tutorials/gymnasium_basics/environment_creation/#sphx-glr-tutorials-gymnasium-basics-environment-creation-py


In [92]:
class AutoBot(gym.Env):

    def __init__(self, render_mode=None, size=5, obstacles_n=1, targets_n=1):
        """
        Ambiente personalizado de Aprendizado por Reforço que simula um agente (AutoBot)
        movendo-se em um grid para alcançar alvos e evitar obstáculos.

        Parâmetros:
        - render_mode (None): Modo de renderização do ambiente.
        - size (int): Tamanho do grid do ambiente.
        - obstacles_n (int): Número de obstáculos no ambiente.
        - targets_n (int): Número de alvos no ambiente.
        """
        self.size = size
        self.window_size = 400
        self.obstacles_n = obstacles_n
        self.obstacles = []
        self.targets_n = targets_n
        self.targets = []
        self.agent_position = None

        # We have 4 actions, corresponding to "right", "up", "left", "down"
        self.action_space = spaces.Discrete(4)
        """
        The following dictionary maps abstract actions from `self.action_space` to
        the direction we will walk in if that action is taken.
        I.e. 0 corresponds to "right", 1 to "up" etc.
        """
        self._action_to_direction = {
            0: np.array([1, 0]),
            1: np.array([0, 1]),
            2: np.array([-1, 0]),
            3: np.array([0, -1]),
        }

    def _get_obs(self):
        return {"agent": self.agent_position,
                "targets": self.targets,
                "obstacles": self.obstacles
                }

    def _get_info(self):
        return {
            "distances": [ np.linalg.norm(self.agent_position - target, ord=1) for target in self.targets ]
        }

    def reset(self, seed=None, options=None):
        """
        Reinicia o ambiente para um novo episódio.

        Parâmetros:
        - seed (int): Semente para a geração de números aleatórios.
        - options (dict): Opções adicionais para resetar o ambiente.

        Retorna:
        - tuple: Uma tupla contendo a observação inicial e informações adicionais.
        """
        super().reset(seed=seed)

        self.targets.clear()
        self.obstacles.clear()

        sampler = spaces.Box(0, self.size-1, shape=(2,), dtype=int)

        self.agent_position = sampler.sample()

        for i in range(self.targets_n):
            target_position = self.agent_position
            while(  np.array_equal( target_position, self.agent_position)):
                target_position = sampler.sample();
            self.targets.append(target_position)

        # TODO: evitar que o obstaculo seja um objetivo
        for i in range(self.obstacles_n):
            obstacles_position = self.agent_position
            while(  np.array_equal( obstacles_position, self.agent_position)):
                obstacles_position = sampler.sample();
            self.obstacles.append(obstacles_position)


        observation = self._get_obs()
        info = self._get_info()

        return observation, info

    def step(self,action):
        """
        Executa uma ação no ambiente e retorna o resultado.

        Parâmetros:
        - action (int): A ação a ser realizada pelo agente.

        Retorna:
        - tuple: Uma tupla contendo a nova observação, recompensa, se o episódio terminou,
                 se o episódio foi truncado e informações adicionais.
        """

        # Em nosso ambiente determinitico a função de transição é se mover
        # Para o proximo estado
        direction = self._action_to_direction[action]
        self.agent_position = np.clip(
            self.agent_position + direction, 0, self.size-1)



        #naive implementation, primeiro para um depois tento fazer para n e para obstaculos
        obs_hit = np.array_equal(self.agent_position, self.obstacles[0])
        terminated = np.array_equal(self.agent_position, self.targets[0])


        # Rewards:
        # 1 Para chegar no objetivo
        # -1 Se bater em um obstaculo
        # -0.01 Para living Penalty
        reward = 1 if terminated else -1 if obs_hit else -0.01

        observation = self._get_obs()
        info = self._get_info()

        return observation, reward, terminated, False, info

    def render(self, mode='human'):
        grid = np.chararray((self.size, self.size))
        grid[:] = "-"
        for obs in self.obstacles:
            grid[obs[0], obs[1]] = "#"  # Marcar obstáculos
        for target in self.targets:
            grid[target[0], target[1]] = "@"  # Marcar pontos de entrega
        grid[self.agent_position[0], self.agent_position[1]] = "X"  # Marcar a posição do agente
        print(grid)




# Definindo a Q-Table

In [93]:
class QLearning:
    def __init__(self, size, n_actions):
        """
        Implementação do algoritmo de Q-Learning para um agente em um ambiente de grid.

        Parâmetros:
        - size (int): Tamanho do grid do ambiente.
        - n_actions (int): Número de ações possíveis no ambiente.
        """
        self.size = size
        self.n_actions = n_actions
        self.q_table = np.zeros((size, size, n_actions))

    def choose_action(self, state, epsilon):
        """
        Escolhe uma ação com base na política atual (exploração ou explotação).

        Parâmetros:
        - state (tuple): O estado atual do agente no ambiente.
        - epsilon (float): Parâmetro que define a probabilidade de escolher uma ação aleatória (exploração).

        Retorna:
        - int: A ação escolhida.
        """
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.choice(self.n_actions)
        else:
            action = np.argmax(self.q_table[state[0], state[1]])
        return action

    def update_q_table(self, state, action, reward, next_state, alpha, gamma):
        """
        Atualiza a tabela Q com base na recompensa recebida e na estimativa do valor futuro.

        Parâmetros:
        - state (tuple): O estado atual do agente.
        - action (int): A ação tomada pelo agente.
        - reward (float): A recompensa recebida pela ação.
        - next_state (tuple): O próximo estado do agente.
        - alpha (float): Taxa de aprendizado.
        - gamma (float): Fator de desconto.
        """
        # Calcula o valor Q máximo para o próximo estado
        max_next_q = np.max(self.q_table[next_state[0], next_state[1]])

        # Atualiza o valor Q para o par estado-ação atual
        #pdb.set_trace()
        self.q_table[state[0], state[1], action] = (1 - alpha) * self.q_table[state[0], state[1], action] + \
                                                    alpha * (reward + gamma * max_next_q)


In [95]:
env = AutoBot()
agent = QLearning(env.size, env.action_space.n)
state = env.reset(42)
epsilon = 0.5
alpha = 0.1
gamma = 0.9

In [96]:
terminated = False
while(not terminated):
    action = agent.choose_action(state[0]["agent"], epsilon)
    print(action)
    step = env.step(action)
    print(f"""
    Ação: {action}
    Obs: {step[0]},
    Reward: {step[1]},
    Terminated: {step[2]},
    Truncated: {step[3]},
    Info: {step[4]}
     """)
    env.render()
    agent.update_q_table(state[0]["agent"], action, step[1], step[0]["agent"], alpha, gamma)
    state = step
    if step[2]:
        terminated = True

0

    Ação: 0
    Obs: {'agent': array([4, 1]), 'targets': [array([0, 1])], 'obstacles': [array([2, 1])]},
    Reward: -0.01,
    Terminated: False,
    Truncated: False,
    Info: {'distances': [4.0]}
     
[[b'-' b'@' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-']
 [b'-' b'#' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-']
 [b'-' b'X' b'-' b'-' b'-']]
1

    Ação: 1
    Obs: {'agent': array([4, 2]), 'targets': [array([0, 1])], 'obstacles': [array([2, 1])]},
    Reward: -0.01,
    Terminated: False,
    Truncated: False,
    Info: {'distances': [5.0]}
     
[[b'-' b'@' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-']
 [b'-' b'#' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'X' b'-' b'-']]
0

    Ação: 0
    Obs: {'agent': array([4, 2]), 'targets': [array([0, 1])], 'obstacles': [array([2, 1])]},
    Reward: -0.01,
    Terminated: False,
    Truncated: False,
    Info: {'distances': [5.0]}
     
[[b'-' b'@' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'-']
 [b'-' b'#' b'-' b'-' b'-']
 [b'-' 

In [97]:
state[0]

{'agent': array([0, 1]),
 'targets': [array([0, 1])],
 'obstacles': [array([2, 1])]}

In [98]:
agent.q_table

array([[[ 0.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ],
        [-0.001     , -0.00109   , -0.001     ,  0.1       ],
        [-0.0019    , -0.0019    , -0.0019    , -0.0019    ],
        [-0.002881  , -0.00199   , -0.001     , -0.001     ]],

       [[-0.001     ,  0.        ,  0.        ,  0.        ],
        [-0.1       , -0.001     ,  0.        , -0.001     ],
        [-0.001     , -0.00199   , -0.001     , -0.001     ],
        [-0.0019    , -0.002071  , -0.00361   , -0.001     ],
        [-0.003529  , -0.002881  , -0.0019    , -0.0019    ]],

       [[-0.002071  , -0.271171  ,  0.        ,  0.        ],
        [-0.00199   , -0.00199   , -0.0019    , -0.001     ],
        [-0.00199   , -0.001     , -0.0019    , -0.1       ],
        [-0.00199   , -0.00271   , -0.00271   , -0.00199   ],
        [-0.003772  , -0.001     , -0.0028    , -0.0037639 ]],

       [[-0.00199   , -0.00199   , -0.00271   , -0.00199   ],
  

In [99]:
politica = np.argmax(agent.q_table, axis=2)

In [100]:
politica

array([[0, 0, 3, 0, 2],
       [1, 2, 0, 3, 2],
       [2, 3, 1, 0, 1],
       [0, 3, 2, 2, 0],
       [1, 0, 2, 0, 1]])